In [6]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns

In [56]:
# import pandas as pd

# # 판다스 데이터프레임(DataFrame)을 출력할 때, 최대 출력할 수 있는 컬럼을 100개로 늘려줍니다.
# # 이렇게 해야 데이터를 분석할 때 출력해서 확인하기 편합니다.
pd.options.display.max_columns = 300
pd.options.display.max_rows = 300

In [19]:
read_file_lst = glob.glob('C:/Users/KIHyuk/Desktop/브런치 데이터/read/*')

In [20]:
exclude_file_lst = ['read.tar']

In [42]:
read_df_lst = []
for f in read_file_lst:
    file_name = os.path.basename(f)
    if file_name in exclude_file_lst:
        print(file_name)
    else:
        df_temp = pd.read_csv(f, header=None, names=['raw'])
        df_temp['dt'] = file_name[:8]
        df_temp['hr'] = file_name[8:10]
        df_temp['user_id'] = df_temp['raw'].str.split(' ').str[0]
        df_temp['article_id'] = df_temp['raw'].str.split(' ').str[1:].str.join(' ').str.strip()
        df_temp['datetime'] = file_name[:4] + "/" + file_name[4:6] + "/" + file_name[6:8] + "/" + file_name[8:10]
        read_df_lst.append(df_temp)

In [43]:
read = pd.concat(read_df_lst)

In [44]:
read.shape

(3507097, 6)

In [45]:
read.head(250)

,raw,dt,hr,user_id,article_id,datetime
0,#e208be4ffea19b1ceb5cea2e3c4dc32c @kty0613_91,20181001,00,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91,2018/10/01/00
1,#0a3d493f3b2318be80f391eaa00bfd1c @miamiyoung_...,20181001,00,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31 @banksalad_49 @rlfrjsdn_95 @rea...,2018/10/01/00
2,#b90d3ee7ed0d7d827aae168e159749f1 @joeunha_4 @...,20181001,00,#b90d3ee7ed0d7d827aae168e159749f1,@joeunha_4 @yoonvi_3,2018/10/01/00
3,#b8b9d09fe2961fd62edc94912bf75a90 @hyejinchoi_...,20181001,00,#b8b9d09fe2961fd62edc94912bf75a90,@hyejinchoi_122 @hyejinchoi_86 @hyejinchoi_42 ...,2018/10/01/00
4,#072f742eda9359cdac03ad080193c11d @doyeonsunim...,20181001,00,#072f742eda9359cdac03ad080193c11d,@doyeonsunim_240 @k52524_297 @bule13_33 @kwong...,2018/10/01/00
...,...,...,...,...,...,...
245,#38fbb915feb3cbe5015ccf6afdd3936e @masism_334 ...,20181001,00,#38fbb915feb3cbe5015ccf6afdd3936e,@masism_334 @wikitree_2604 @masism_334 @2hanan...,2018/10/01/00
246,#c5dbc3fed1c8c1c52e5cb2065a0d4b69 @jonnaalive_...,20181001,00,#c5dbc3fed1c8c1c52e5cb2065a0d4b69,@jonnaalive_288 @jonnaalive_288 @bookguru_29 @...,2018/10/01/00
247,#13b3009a8698e9d5e892534d9dcdac62 @onthepaper_...,20181001,00,#13b3009a8698e9d5e892534d9dcdac62,@onthepaper_51 @onthepaper_52 @onthepaper_51 @...,2018/10/01/00
248,#ceaa727b363eb14a913bb5292b054894 @dryjshin_22...,20181001,00,#ceaa727b363eb14a913bb5292b054894,@dryjshin_227 @yjwoo_66 @syc1025_72,2018/10/01/00


In [46]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(' ')))

In [47]:
read_cnt_by_user = read['article_id'].str.split(' ').map(len)

In [48]:
read_raw = pd.DataFrame({'dt': np.repeat(read['dt'], read_cnt_by_user),
                         'hr': np.repeat(read['hr'], read_cnt_by_user),
                         'datetime' : np.repeat(read['datetime'],read_cnt_by_user),
                         'user_id': np.repeat(read['user_id'], read_cnt_by_user),
                         'article_id': chainer(read['article_id'])})

In [49]:
read_raw.shape

(22110706, 5)

In [36]:
print("전체 데이터 건수:", read_raw.shape)
print("중복 소비를 제외한 데이터 건수:", read_raw[['user_id', 'article_id']].drop_duplicates().shape)
print("Unique 독자 수:", len(read_raw['user_id'].unique()))
print("소비된 Unique 글 수:", len(read_raw['article_id'].unique()))

전체 데이터 건수: (22110706, 4)
중복 소비를 제외한 데이터 건수: (12597878, 2)
Unique 독자 수: 306222
소비된 Unique 글 수: 505841


In [55]:
read_raw.to_csv('read_raw')

In [51]:
# read_raw = pd.read_csv('read_raw',index_col='datetime', parse_dates=True)

In [52]:
# read_raw

,Unnamed: 0,dt,hr,user_id,article_id
datetime,,,,,
2018-10-01 00:00:00,0,20181001,0,#e208be4ffea19b1ceb5cea2e3c4dc32c,@kty0613_91
2018-10-01 00:00:00,1,20181001,0,#0a3d493f3b2318be80f391eaa00bfd1c,@miamiyoung_31
2018-10-01 00:00:00,1,20181001,0,#0a3d493f3b2318be80f391eaa00bfd1c,@banksalad_49
2018-10-01 00:00:00,1,20181001,0,#0a3d493f3b2318be80f391eaa00bfd1c,@rlfrjsdn_95
2018-10-01 00:00:00,1,20181001,0,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140
...,...,...,...,...,...
2019-02-28 23:00:00,1229,20190228,23,#3eec960b2ad12fc41ec986032effc8b2,@leewoosview_189
2019-02-28 23:00:00,1230,20190228,23,#1eab0886c0f0f32156f9ab1e5d0fffab,@rory_7
2019-02-28 23:00:00,1230,20190228,23,#1eab0886c0f0f32156f9ab1e5d0fffab,@rory_7


In [54]:
# del read_raw['Unnamed: 0']